In [ ]:
import cv2
import numpy as np
import cairocffi as cairo
import io
import preproc
import detection
from detection.util import CHARTYPE
from chrecog.predict import get_session, load_ckpt, get_pred_one, get_candidate
sess = get_session()
load_ckpt(sess, "data/ckpt/161103_2ID.ckpt")
import reconst
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'Greys'

In [ ]:
def write_line(text, font="NanumGothic"):
    WIDTH, HEIGHT = 540, 100
    surface = cairo.ImageSurface (cairo.FORMAT_RGB24, WIDTH, HEIGHT)
    ctx = cairo.Context (surface)
    ctx.set_font_size(40)
    ctx.set_source_rgb(0, 0, 0)
    ctx.paint()
    ctx.set_source_rgb(1, 1, 1)
    ctx.select_font_face(font, cairo.FONT_SLANT_NORMAL,
                cairo.FONT_WEIGHT_NORMAL)
    ctx.move_to(20,40)
    ctx.show_text(text)
    fb = io.BytesIO()
    surface.write_to_png(fb)
    fb.seek(0)
    blob = fb.read()
    blob_array = np.asarray(bytearray(blob), dtype=np.uint8)
    mat = cv2.imdecode(blob_array, 0)
    fb.close()
    return mat

In [ ]:
def test_line(text, font="NanumGothic"):
    img = write_line(text, font)
    plt.figure()
    plt.imshow(img)

    processed = preproc.process(img)
    graphs = detection.get_graphs(processed)

    i=0
    for p in graphs:
        for l in p.lines:
            for c in l.chars:
                if c.type == CHARTYPE.BLANK:
                    c.value = " "
                    continue
                if i % 10 == 0:
                    plt.figure()
                plt.subplot(1,10,i%10+1)
                plt.xticks(())
                plt.yticks(())
                plt.imshow(c.img)
                i += 1
                c.pred = get_pred_one(c.img / 255)
                c.value = get_candidate(c.pred)

    print(reconst.build_graphs(graphs), end="")
    
def test_img(path):
    img = cv2.imread(path)
    plt.figure()
    plt.imshow(img)

    processed = preproc.process(img)
    graphs = detection.get_graphs(processed)

    i=0
    j=0
    plt.figure()
    for p in graphs:
        for l in p.lines:
            for c in l.chars:
                if c.type == CHARTYPE.BLANK:
                    c.value = " "
                    continue
                assert c.type == CHARTYPE.CHAR
                if j > 10:
                    j = 0
                    i += 1
                plt.subplot(10,10, i*10 + j + 1)
                plt.xticks(())
                plt.yticks(())
                plt.imshow(c.img)
                j += 1
                if i*10 + j + 1 > 100:
                    i = 0
                    j = 0
                    plt.figure()
                c.pred = get_pred_one(c.img / 255)
                c.value = get_candidate(c.pred)

    print(reconst.build_graphs(graphs))

In [ ]:
test_img("data/sample_sa.png")
#test_line("오. 왜 그러세요...")
#test_line("컴퓨터공학과입니다.")